In [1]:
import pandas as pd

In [2]:
%load_ext rpy2.ipython

# Question Difficulty Analysis

## Select course and load data set

In [3]:
data_dir = '/Users/benny/data/L@S_2021'
course = 'microbiology'

In [4]:
data_set_filename = f'difficulty_{course}.txt'
data_set = pd.read_csv( f'{data_dir}/{data_set_filename}', sep='\t' )

In [5]:
data_set

,student,question,course_page_number,unit_page_number,module_page_number,page_question_number,question_type,answered,correct
0,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135e55,18,16,5,0,AG_FITB,1,1
1,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135e77,18,16,5,1,AG_FITB,1,0
2,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135e9d,18,16,5,2,AG_Matching,1,1
3,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135ead,18,16,5,3,AG_Matching,1,1
4,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135ebd,18,16,5,4,AG_Matching,1,1
...,...,...,...,...,...,...,...,...,...
42109,600e1302e16a3a3764c6730a,5f2e4e80e3a4d9432b13acb0,31,29,9,1,AG_Matching,1,1
42110,600e1302e16a3a3764c6730a,5f2e4e80e3a4d9432b13acc4,31,29,9,2,AG_Matching,1,1
42111,600e1302e16a3a3764c6730a,5f2e4e80e3a4d9432b13acee,31,29,9,3,AG_Matching,1,1
42112,600e1302e16a3a3764c6730a,5f2e4e80e3a4d9432b13ad02,31,29,9,4,AG_Matching,1,1


## Mean difficulty

In [6]:
data_set.groupby( 'question_type' ).mean().sort_values( by='correct', ascending=False )

,course_page_number,unit_page_number,module_page_number,page_question_number,answered,correct
question_type,,,,,,
AG_Matching,49.850411,47.850411,4.066796,4.719844,1.0,0.905174
HA_MC,40.181774,38.181774,4.168040,8.653471,1.0,0.695912
HA_Pulldown,53.014690,51.014690,5.681357,19.152778,1.0,0.682425
AG_FITB,44.206619,42.206619,4.347276,4.846932,1.0,0.627902
HA_DnD_Image,54.000000,52.000000,6.000000,26.542411,1.0,0.618304
HA_DnD_Table,42.097065,40.097065,7.880361,11.828442,1.0,0.566591
HA_MCMS,42.247438,40.247438,5.026354,10.193265,1.0,0.433382


## Regression model

In [7]:
%%R
library( lme4 )

R[write to console]: Loading required package: Matrix



Standardize the continuous variables.

In [8]:
for col in [ 'course_page_number', 'unit_page_number', 'module_page_number', 'page_question_number' ]:
    data_set[ col ] = ( data_set[ col ] - data_set[ col ].mean() ) / data_set[ col ].std()

In [9]:
data_set.to_csv( '/tmp/to_r.csv', index=False )

In [10]:
%%R
df <- read.csv( '/tmp/to_r.csv' )

In [11]:
%%R
lme.model <- glmer( correct ~ course_page_number + unit_page_number + module_page_number + page_question_number + question_type
                            + (1|student) + (1|question), family=binomial(link=logit),
                    data=df,
                    control=glmerControl( optimizer="bobyqa", optCtrl=list(maxfun=2e4) ) )
summary( lme.model )

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
correct ~ course_page_number + unit_page_number + module_page_number +  
    page_question_number + question_type + (1 | student) + (1 |      question)
   Data: df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 20000))

     AIC      BIC   logLik deviance df.resid 
 37093.6  37197.4 -18534.8  37069.6    42102 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.9940  0.0762  0.2615  0.4930  5.9037 

Random effects:
 Groups   Name        Variance Std.Dev.
 question (Intercept) 1.5298   1.2369  
 student  (Intercept) 0.2855   0.5343  
Number of obs: 42114, groups:  question, 1023; student, 96

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                0.51101    0.14096   3.625 0.000289 ***
course_page_number        -0.13900    0.04155  -3.345 0.000822 ***
module_page_number 